In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
AIDEVS_API_KEY = os.environ["AIDEVS_API_KEY"]

In [3]:
client = OpenAI()

In [4]:
response = requests.post("https://tasks.aidevs.pl/token/whisper", json=dict(apikey=AIDEVS_API_KEY))
response_json = json.loads(response.text)
token = response_json["token"]

response = requests.get(f"https://tasks.aidevs.pl/task/{token}")
response_json = json.loads(response.text)

print(response_json)

{'code': 0, 'msg': 'please return transcription of this file: https://tasks.aidevs.pl/data/mateusz.mp3', 'hint': 'use WHISPER model - https://platform.openai.com/docs/guides/speech-to-text'}


In [5]:
mp3_url = response_json["msg"].split("please return transcription of this file: ")[1]
response = requests.get(mp3_url, params={"downloadformat": "mp3"})

if response.ok:
    with open("task-mp3-file.mp3", mode="wb") as file:
        file.write(response.content)
else:
    print(f"Response error, status code: {response.status_code}")

In [6]:
with open("task-mp3-file.mp3", mode="rb") as file:
    transcription = client.audio.transcriptions.create(
      model="whisper-1", 
      file=file, 
      response_format="text"
    )
print(transcription)

Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?



In [7]:
response = requests.post(f"https://tasks.aidevs.pl/answer/{token}", json=dict(answer=transcription))
print(response.text)

{
    "code": 0,
    "msg": "OK",
    "note": "CORRECT"
}
